# Tuberculosis Diagnosis using VGG16 based feature extractor and a custom classifier
This notebook is written and executed by **Dr Raheel Siddiqi** on *13-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 10-fold cross validation is used to evaluate the model.

## TensorFlow and Keras version used

In [1]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.14.0
Keras Version:  2.2.4-tf


## Model Setup

In [2]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model

In [3]:
image_height = 100
image_width = 100
batch_size = 4
no_of_epochs  = 50

## Dataset Preparation

In [4]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
%matplotlib inline

DATADIR = '/kaggle/input/pulmonary-chest-xray-abnormalities/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png'
data=[]
for img in tqdm(os.listdir(DATADIR)):
    try:
        img_array = cv2.imread(os.path.join(DATADIR,img))
        img_array = cv2.resize(img_array, (image_height, image_width))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
        img_array = img_array.astype(np.float32)/255.
        if img[-5]=='0':
            data.append([img_array, 0])
        else:
            data.append([img_array, 1])
    except Exception as e:   
            pass
print(len(data))

100%|██████████| 663/663 [01:18<00:00,  8.48it/s]

662


In [5]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

1
0
1
1
0
1
0
0
1
0


In [6]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


## 10-Fold Cross Validation

In [7]:
k=10
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=batch_size,epochs=no_of_epochs) # 50 epochs per model
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])

58892288/58889256 [==============================] - 2s 0us/step
Epoch 1/50
596/596 [==============================] - 6s 10ms/sample - loss: 0.6379 - acc: 0.6393
Epoch 2/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.5730 - acc: 0.7383
Epoch 3/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.5421 - acc: 0.7601
Epoch 4/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.4805 - acc: 0.8188
Epoch 5/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.4450 - acc: 0.8205
Epoch 6/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.4118 - acc: 0.8423
Epoch 7/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.3893 - acc: 0.8507
Epoch 8/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.4019 - acc: 0.8490
Epoch 9/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.3466 - acc: 0.8591
Epoch 10/50
596/596 [==============================] - 3s 6ms

596/596 [==============================] - 3s 6ms/sample - loss: 0.2757 - acc: 0.8893
Epoch 11/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.2913 - acc: 0.8758
Epoch 12/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.2406 - acc: 0.9060
Epoch 13/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.1935 - acc: 0.9211
Epoch 14/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.1870 - acc: 0.9329
Epoch 15/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.1011 - acc: 0.9614
Epoch 16/50
596/596 [==============================] - 4s 6ms/sample - loss: 0.1359 - acc: 0.9446
Epoch 17/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.0899 - acc: 0.9664
Epoch 18/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.0886 - acc: 0.9698
Epoch 19/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.0079 - acc: 1.0000
Epoch 20/50
596/596 [===========

596/596 [==============================] - 4s 6ms/sample - loss: 1.5213e-07 - acc: 1.0000
Epoch 42/50
596/596 [==============================] - 4s 6ms/sample - loss: 1.3312e-07 - acc: 1.0000
Epoch 43/50
596/596 [==============================] - 4s 6ms/sample - loss: 1.1780e-07 - acc: 1.0000
Epoch 44/50
596/596 [==============================] - 4s 6ms/sample - loss: 1.0376e-07 - acc: 1.0000
Epoch 45/50
596/596 [==============================] - 4s 6ms/sample - loss: 9.2585e-08 - acc: 1.0000
Epoch 46/50
596/596 [==============================] - 4s 6ms/sample - loss: 8.3169e-08 - acc: 1.0000
Epoch 47/50
596/596 [==============================] - 4s 6ms/sample - loss: 7.4351e-08 - acc: 1.0000
Epoch 48/50
596/596 [==============================] - 4s 6ms/sample - loss: 6.6715e-08 - acc: 1.0000
Epoch 49/50
596/596 [==============================] - 4s 6ms/sample - loss: 6.0757e-08 - acc: 1.0000
Epoch 50/50
66/66 [==============================] - 0s 5ms/sample - loss: 3.1799 - acc: 0.818

66/66 [==============================] - 0s 6ms/sample - loss: 1.5212 - acc: 0.8939
Epoch 1/50
596/596 [==============================] - 4s 7ms/sample - loss: 0.7067 - acc: 0.5302
Epoch 2/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.6787 - acc: 0.5856
Epoch 3/50
596/596 [==============================] - 4s 6ms/sample - loss: 0.6940 - acc: 0.5336
Epoch 4/50
596/596 [==============================] - 4s 6ms/sample - loss: 0.6950 - acc: 0.4966
Epoch 5/50
596/596 [==============================] - 4s 6ms/sample - loss: 0.6933 - acc: 0.5235
Epoch 6/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.6929 - acc: 0.5101
Epoch 7/50
596/596 [==============================] - 4s 6ms/sample - loss: 0.6941 - acc: 0.5218
Epoch 8/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.6927 - acc: 0.5235
Epoch 9/50
596/596 [==============================] - 3s 6ms/sample - loss: 0.6930 - acc: 0.5134
Epoch 10/50
596/596 [======================

## Average Validation Score

In [8]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.8060606
